# Despacho múltiplo

Neste notebook, vamos explorar o conceito de [despacho múltiplo](https://en.wikipedia.org/wiki/Multiple_dispatch), que é um recurso chave da linguagem Julia.

Para entender despacho múltiplo em Julia, vamos começar com um exemplo que nós já vimos.

Em Julia, podemos declarar funções sem informar o tipo dos argumentos da função.
Por exemplo:

In [1]:
f(x) = x^2

f (generic function with 1 method)

Então a Julia vai determinar automaticamente sobre quais tipos de argumentos faz sentido aplicar a função, e sobre quais tipos não faz:

In [2]:
f(9)

81

In [3]:
f([1, 2, 3])

LoadError: MethodError: no method matching ^(::Vector{Int64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at strings/basic.jl:730
[0m  ^([91m::Complex{<:AbstractFloat}[39m, ::Integer) at complex.jl:860
[0m  ^([91m::Complex{<:Integer}[39m, ::Integer) at complex.jl:862
[0m  ...

## Especificando os tipos dos argumentos

Temos a opção de declarar explicitamente quais tipos de argumentos uma função pode receber.
Por exemplo, vamos definir uma função `teste` que só pode receber argumentos do tipo `String`:

In [4]:
teste(x::String, y::String) = println("Os argumentos x e y são strings")

teste (generic function with 1 method)

Vamos verificar que a função `teste` age sobre strings, mas não age sobre argumentos de outros tipos:

In [5]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


In [6]:
teste(2, 3)

LoadError: MethodError: no method matching teste(::Int64, ::Int64)

Para fazer com que a função `teste` aceite números inteiros, vamos definir `teste` para argumentos do tipo `Int`:

In [7]:
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros")

teste (generic function with 2 methods)

In [8]:
teste(2, 3)

Os argumentos x e y são números inteiros


Agora `teste` age sobre os inteiros.
Mas observe, a função `teste` continua definida quando `x` e `y` são strings:

In [9]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


Quando declaramos
```julia
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros"),
```
não removemos ou substituímos
```julia
teste(x::String, y::String),
```
simplesmente adicionamos um método à função genérica `teste`.

Uma **função genérica** é um conceito abstrato associado a uma operação.
Por exemplo, a função genérica `+` representa o conceito de adição.
Um **método** é uma implementação específica de uma função genérica para um tipo específico de argumento.
Por exemplo, a função `+` possui métodos para números de precisão flutuante, inteiros, matrizes, etc.
Podemos usar `methods` para ver os métodos que a função `teste` possui:

In [10]:
methods(teste)

# 2 methods for generic function "teste":
[1] teste(x::String, y::String) in Main at In[4]:1
[2] teste(x::Int64, y::Int64) in Main at In[7]:1

E podemos ver os métodos da função `+`:

In [11]:
methods(+)

# 206 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383
[3] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:531
[4] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:537
[5] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:414
[8] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/periods.jl:378
[9] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Hermitian) in LinearAlgebra at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:469
[10] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.SymTridiagonal{var"#s882", V} where {var"#s882"<:Real, V<:AbstractVector{var"#s882"}}) in LinearAlgebra at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:488
[11] +(H::LinearAlgebra.Hermitian, D::LinearAlgebra.Diagonal{var"#s885", V} where {var"#s885"<:Real, V<:AbstractVector{var"#s885"}}) in LinearAlgebra at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:197
[12] +(A::LinearAlgebra.Hermitian, J::LinearAlgebra.UniformScaling{<:Complex}) in LinearAlgebra at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:195
[13] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:15
[14] +(A::LinearAlgebra.Hermitian, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:18
[15] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Symmetric{<:Real, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:26
[16] +(A::LinearAlgebra.Hermitian, B::LinearAlgebra.Symmetric{var"#s885", S} where {var"#s885"<:Real, S<:(AbstractMatrix{<:var"#s885"})}) in LinearAlgebra at /usr/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:483
[17] +(A::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:24
[18] +(x::P, y::P) where P<:Dates.Period in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/periods.jl:77
[19] +(x::Dates.Period, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/periods.jl:367
[20] +(y::Dates.Period, x::Dates.CompoundPeriod) in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/periods.jl:369
[21] +(y::Dates.Period, x::Dates.TimeType) in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/arithmetic.jl:85
[22] +(x::Dates.Period, r::AbstractRange{<:Dates.TimeType}) in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/ranges.jl:63
[23] +(y::Union{Dates.CompoundPeriod, Dates.Period}, x::AbstractArray{<:Dates.TimeType}) in Dates at /usr/share/julia/stdlib/v1.8/Dates/src/deprecated.jl:14
[24] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:14
[25] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Hermitian) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:17
[26] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:14
[27] +(A::SparseArrays.AbstractSparseMatrix, B::LinearAlgebra.Symmetric) in SparseArrays at /usr/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:17
[28] +(A::LinearA

Portanto, agora podemos aplicar `teste` a strings e inteiros.
Quando aplicamos `teste` a um par de argumentos, a Julia vai inferir os tipos dos argumentos e despachar o método apropriado.
Isso é **despacho múltiplo**.
Despacho múltiplo torna os programas genéricos e rápidos.

Para ver qual método está sendo despachado quando chamamos uma função genérica, usamos a macro `@which`:

In [12]:
@which teste(2, 3)

teste(x::Int64, y::Int64) in Main at In[7]:1

In [13]:
@which 3.0 + 5.0

+(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383